In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18146969831970218149, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11631932619882281736
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 16
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)
epochs = 50

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [7]:
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.noise import AlphaDropout


def Unet(img_size):
    inputs = Input((img_size, img_size, 3))
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = AlphaDropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = AlphaDropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = AlphaDropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = AlphaDropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = AlphaDropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = AlphaDropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = AlphaDropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = AlphaDropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = AlphaDropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [8]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [9]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [10]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 1632
predict_size_validation: 544
predict_size_test: 2


In [11]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [12]:
model = Unet(img_size = 256)
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
alpha_dropout_1 (AlphaDropout)  (None, 256, 256, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [13]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/50
1632/1632 [==============================] - 374s 229ms/step - loss: -0.1345 - dice_coef: 0.1345 - acc: 0.5043 - mean_squared_error: 0.3393 - Jaccard_index: 0.0725 - Specificity: 0.9683 - Sensitivity: 1.3587 - precision: 0.0847 - recall: 0.6170 - f1score: 0.1464 - val_loss: -0.1851 - val_dice_coef: 0.1851 - val_acc: 0.6312 - val_mean_squared_error: 0.2071 - val_Jaccard_index: 0.1028 - val_Specificity: 0.9582 - val_Sensitivity: 1.1989 - val_precision: 0.1274 - val_recall: 0.5210 - val_f1score: 0.2033error: 0.3840 - Jaccard_index: 0.0647 - Specificity: - ET - ETA: 58s - loss: -0.1303 - dice_coef: 0.1303 - acc - ETA: 1s - loss: -0.1343 - dice_coef: 0.1343 - acc: 0.5036 - mean_squared_error: 0.3395 - Jaccard_index: 0.0724 - Specificity: 0.9683 - Sensitivity: 1.3592 - precision: 0.0846 - recall: 0.6171
Epoch 2/50
1632/1632 [==============================] - 370s 227ms/step - loss: -0.1517 - dice_coef: 0.1517 - acc: 0.6155 - mean_squared_error: 0.2637 - Jaccard_index: 0.0824 - Spe

1632/1632 [==============================] - 374s 229ms/step - loss: -0.1655 - dice_coef: 0.1655 - acc: 0.7129 - mean_squared_error: 0.1614 - Jaccard_index: 0.0909 - Specificity: 0.9617 - Sensitivity: 1.1203 - precision: 0.1220 - recall: 0.3654 - f1score: 0.1819 - val_loss: -0.1995 - val_dice_coef: 0.1995 - val_acc: 0.6948 - val_mean_squared_error: 0.1770 - val_Jaccard_index: 0.1122 - val_Specificity: 0.9548 - val_Sensitivity: 1.1312 - val_precision: 0.1497 - val_recall: 0.4114 - val_f1score: 0.2176
Epoch 10/50
1632/1632 [==============================] - 372s 228ms/step - loss: -0.1654 - dice_coef: 0.1654 - acc: 0.7130 - mean_squared_error: 0.1615 - Jaccard_index: 0.0908 - Specificity: 0.9617 - Sensitivity: 1.1202 - precision: 0.1221 - recall: 0.3647 - f1score: 0.1818 - val_loss: -0.1985 - val_dice_coef: 0.1985 - val_acc: 0.6777 - val_mean_squared_error: 0.1949 - val_Jaccard_index: 0.1115 - val_Specificity: 0.9558 - val_Sensitivity: 1.1525 - val_precision: 0.1445 - val_recall: 0.4512 

1632/1632 [==============================] - 370s 227ms/step - loss: -0.1657 - dice_coef: 0.1657 - acc: 0.7144 - mean_squared_error: 0.1603 - Jaccard_index: 0.0910 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1225 - recall: 0.3625 - f1score: 0.1821 - val_loss: -0.1991 - val_dice_coef: 0.1991 - val_acc: 0.7050 - val_mean_squared_error: 0.1664 - val_Jaccard_index: 0.1119 - val_Specificity: 0.9540 - val_Sensitivity: 1.1181 - val_precision: 0.1525 - val_recall: 0.3880 - val_f1score: 0.2170ean_squared_error: 0.1604 - Jaccard_index: 0.0909 - Specificity: 0.9616 - Sensitivity: 1.1185 - pr - ETA: 47s - loss: -0.1658 - dice_coef: 0.1658 - acc: 0.7135 - mean_squared_error: 0.1603 - Jaccard_index: 0.0911 - ETA: 1s - loss: -0.1658 - dice_coef: 0.1658 - acc: 0.7145 - mean_squared_error: 0.1602 - Jaccard_index: 0.0911 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1226 - recall: 0.
Epoch 17/50
1632/1632 [==============================] - 375s 230ms/step - loss: -0.1657 - di

1632/1632 [==============================] - 373s 228ms/step - loss: -0.1657 - dice_coef: 0.1657 - acc: 0.7144 - mean_squared_error: 0.1603 - Jaccard_index: 0.0910 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1225 - recall: 0.3627 - f1score: 0.1821 - val_loss: -0.1982 - val_dice_coef: 0.1982 - val_acc: 0.7098 - val_mean_squared_error: 0.1612 - val_Jaccard_index: 0.1114 - val_Specificity: 0.9537 - val_Sensitivity: 1.1120 - val_precision: 0.1537 - val_recall: 0.3752 - val_f1score: 0.2159 - Specificity: 0.9617 - Sensitivity: 1.1186 - precision: 0.1227 - recall: 0.3635 - f1score: 0 - ETA: 49s - loss: -0.1658 - dice_coef: 0.1658 - acc: 0.7143 - mean_squared_error: 0.1602 - Jaccard_index: 0.0911 - Specificity: 0.9617 - Sensitivity: 1.1186 - precision: 0.1228 - recall: 0.3636 - f1sc - ETA: 47s - loss: -0.1658 - dice_coef: 0.1658 - acc: 0.7143 - mean_squared_error: 0.1602 - Ja
Epoch 25/50
1632/1632 [==============================] - 373s 228ms/step - loss: -0.1656 - dice_coef: 0.

1632/1632 [==============================] - 372s 228ms/step - loss: -0.1656 - dice_coef: 0.1656 - acc: 0.7144 - mean_squared_error: 0.1603 - Jaccard_index: 0.0909 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1225 - recall: 0.3619 - f1score: 0.1820 - val_loss: -0.1743 - val_dice_coef: 0.1743 - val_acc: 0.6762 - val_mean_squared_error: 0.1982 - val_Jaccard_index: 0.0964 - val_Specificity: 0.9541 - val_Sensitivity: 1.1510 - val_precision: 0.1271 - val_recall: 0.3902 - val_f1score: 0.1901.9615 - Sensitivity: 1.1184 - precisio - ETA: 1:45 - loss: -0.1660 - dice_ - ETA: 1:29 - loss: -0.1656 - dice_coef: 0.1656 - acc: 0.7179 - mean_squared_error: 0.1604 - Jaccard_in - ETA: 1:20 - loss: -0.1658 - dice_coef: 0.1658 - acc: 0.7175 - mean_squared_error: 0.1603 - Jaccard_index: 0.0910 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1227 - recall: 0.3618 - ETA: 1:18 - loss: -0.1659 - dice_coef: 0.1659 - acc: 0.7173 - mean_squared_error: 0.1603 - Jaccard_index: 0.0911 - Spec

1632/1632 [==============================] - 372s 228ms/step - loss: -0.1656 - dice_coef: 0.1656 - acc: 0.7144 - mean_squared_error: 0.1603 - Jaccard_index: 0.0909 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1225 - recall: 0.3619 - f1score: 0.1820 - val_loss: -0.1980 - val_dice_coef: 0.1980 - val_acc: 0.7101 - val_mean_squared_error: 0.1611 - val_Jaccard_index: 0.1113 - val_Specificity: 0.9536 - val_Sensitivity: 1.1115 - val_precision: 0.1537 - val_recall: 0.3737 - val_f1score: 0.2157uared_error: 0.1599 - Jaccard_index: 0.0926 - Specificity: 0.961 - ETA: 2:05 - loss: -0.1686 - dice_coef: 0.1686 - acc: 0.7138 - mean_squared_error: 0.1599 - Jaccard_index: 0.0927 - Specificity: 0.9615 - Sensitivity: 1.1182 - precision: 0.1247 - recall: 0.36 - ETA: 2:04 - loss: -0.1685 - dice_coef: 0.1685 - acc: 0.7137 - mean_squared_error: 0.1599 - Jaccard_index: 0.0927 - Specificity: 0.9616 - Sensitivity: 1.1182 - precision: 0.1246 - recall: 0.3666 - f1s - ETA: 2:03 - loss: -0.1687 - dice_

1632/1632 [==============================] - 374s 229ms/step - loss: -0.1657 - dice_coef: 0.1657 - acc: 0.7144 - mean_squared_error: 0.1603 - Jaccard_index: 0.0910 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1225 - recall: 0.3624 - f1score: 0.1821 - val_loss: -0.1765 - val_dice_coef: 0.1765 - val_acc: 0.6814 - val_mean_squared_error: 0.1921 - val_Jaccard_index: 0.0977 - val_Specificity: 0.9540 - val_Sensitivity: 1.1453 - val_precision: 0.1299 - val_recall: 0.3839 - val_f1score: 0.1925r: 0.160 - ETA: 2:12 - loss: -0.1646 - dice_coef: 0.1646 - acc: 0.7157 - mean_squared_error: 0.1604 - Jaccard_index: 0.0903 - Specificity: 0.9617 - Sensit - ETA: 2:07 - loss: -0.1648 - dice_coef: 0.1648 - acc: 0.7154 - mean_squared_error: 0.1603 - Jaccard_index: 0.0905 - Specificity: 0.9617 - Sensitivity: 1.1186 - precision: 0.1220 - recall: 0.36 - ETA: 2:05 - loss: -0.1650 - dice_coef: 0.1650 - acc: 0.7156 - mean_squared_error: 0.1603 - Jaccard_index: 0.0906 - Specif - ETA: 1:57 - loss: -0.

1632/1632 [==============================] - 371s 227ms/step - loss: -0.1656 - dice_coef: 0.1656 - acc: 0.7144 - mean_squared_error: 0.1603 - Jaccard_index: 0.0910 - Specificity: 0.9616 - Sensitivity: 1.1185 - precision: 0.1225 - recall: 0.3622 - f1score: 0.1820 - val_loss: -0.1868 - val_dice_coef: 0.1868 - val_acc: 0.6963 - val_mean_squared_error: 0.1756 - val_Jaccard_index: 0.1041 - val_Specificity: 0.9538 - val_Sensitivity: 1.1274 - val_precision: 0.1408 - val_recall: 0.3785 - val_f1score: 0.2035


In [14]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

Validation Score:  -0.1943027137219906
Validation Accuracy:  0.1943027137219906


In [15]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

28/28 [==============================] - 5s 178ms/step


In [16]:
prediction.shape

(392, 256, 256, 1)

In [17]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

['loss', 'dice_coef', 'acc', 'mean_squared_error', 'Jaccard_index', 'Specificity', 'Sensitivity', 'precision', 'recall', 'f1score']
[-0.1855670329289777, 0.18556703312056405, 0.8219895730699812, 0.17691566228866576, 0.10298631981015205, 0.9604331489971706, 1.1338796697344098, 0.12053534224629402, 0.40973154510770526, 0.1855287206385817]


In [18]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

Training co-effiency    : 0.09101687830772397;
Validation co-effiency : 0.11224776216926437


In [ ]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()